<a href="https://colab.research.google.com/github/nicole-sb/atari-HEAD/blob/main/pacman_attention_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
h,w = 160,210
hidden_size = 256
batch_size = 16

In [ ]:
class Mask(torch.nn.Module):
  def  __init__(self):
      super().__init__()
      self.MLP = torch.nn.Sequential(
          torch.nn.Linear(in_features=1000, out_features =  64),
          torch.nn.ReLU(),
          torch.nn.Linear(in_features=65, out_features=h*w*1) #
      )
      self.sigmoid = torch.nn.Sigmoid()

      def forward(self, random_vector, gaze_bias):
        #start we have x with shape (1000,)
        #apply our fully conncted layer
        out = self.MLP(random_vector)
        out = out.view() #unflatten
        #apply gaze_bias to learned mask
        out = out + gaze_bias
        #apply sigmoid now to make values go between 0 and 1
        return self.sigmoid(out)

class CNN(torch.nn.Module):
  def  __init__(self):
      super().__init__()
      
      self.learned_mask = Mask()

      self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=100, kernel_size=3, stride=1)
      self.relu = torch.nn.ReLU()
      self.maxpool1 = torch.nn.MaxPool2d(kernel_size=2)

      self.flatten = torch.nn.Flatten()
      self.MLP = torch.nn.Sequential(
          torch.nn.Linear(in_features = 100*h*w, out_features=16),
          torch.nn.ReLU(),
          torch.nn.Linear(in_features=16, out_features=9)
      )
      self.final_activation = torch.nn.Softmax()
 
  def forward(self, x, random_vector, mask):
    """This applies our input image with the mask and then runs it"""

    learned_weight_mask = self.learned.mask(random_vector)

    
    #Apply the mask to the image to get initial input to CNN parts
    out = torch.mul(x,mask) #shape (3,h,w)

    out = self.conv1(out) #shape (100,h,w)
    out = self.relu1(out)
    out = self.maxpool1(out)

    #apply the f
    out = self.flatten(out)

    out = self.final_activation(self.MLP(out)) #this will get you probability vector with probs for each class

In [ ]:
num_epochs = 17814
images = None #TODO: load_images
cnn_model =  CNN()
loss_func = torch.nn.CrossEntropyLoss()
torch.optim.SGD(cnn_model.parameters(),lr=1e-2, momentum=0.9)

for epoch in range(num_epochs):
  for image, gaze_coords, y in training_data:
    #every step
   

    gaze_x, gaze_y = gaze_coords[0], gaze_coords[1]


    #make one hot encoded gaze_bias
    gaze_bias = torch.zeros((1,h,w))
    gaze_bias[:,gaze_y, gaze_x] = 1

    #make random vector that will be used to make mask
    random_vector = torch.rand(1000)

    #zero gradients before calculating gradients of loss with respect to the image
    opt_cnn.zero_grad()

    #pass image into cnn_model
    y_hat = cnn_model(image, gaze_bias)

    #calculate loss
    loss = loss_func(y_hat, y)

    #calculate gradients with backprob
    loss.backward()

    #step optimizer
    opt_cnn.step()

SyntaxError: ignored